In [ ]:
from selenium.webdriver.chrome import webdriver, options
from selenium.webdriver.remote.webelement import WebElement
from selenium.webdriver.support.relative_locator import By 
from selenium.webdriver.support.select import Select 
from selenium.common.exceptions import NoSuchElementException, ElementNotInteractableException, StaleElementReferenceException
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.support import expected_conditions as EC
from urllib.parse import urlsplit
import re
import os
import pandas as pd
from selenium.webdriver.support.color import Color
import time
import traceback


class ProductType:
    SINGLE = 'single'
    MULTI_SIZE = 'multi-size'
    MULTI_COLOR = 'multi-color'
    MULTI_SHADE = 'multi-shade'

def safe_get_element(wd: webdriver.WebDriver, by: By, value:str):
    try:
        element = wd.find_element(by, value)
        return element
    except NoSuchElementException:
        return None
    

def click_element_refresh_stale(wd: webdriver.WebDriver, element: WebElement, locator: tuple[By, str], index = None):
    while True:
        try:
            wd.execute_script("arguments[0].click();", element)
            return element
        except StaleElementReferenceException:
            print('Could not click element. Refreshing...')
            if index is None:
                element = wd.find_element(locator)
            else:
                element = wd.find_elements(locator)[index]
        

def get_variation_images(wd: webdriver.WebDriver, variation_details:dict[str, object]):
    right_arrow = wd.find_element(By.CLASS_NAME, 'athenaProductImageCarousel_rightArrow')
    for i, image in enumerate(wd.find_elements(By.CLASS_NAME, 'athenaProductImageCarousel_image')):
        if i != 0:
            right_arrow = click_element_refresh_stale(wd, right_arrow, ('class name', 'athenaProductImageCarousel_rightArrow'))
        while True:
            try:
                image_src = image.get_attribute('src')
                break
            except StaleElementReferenceException:
                print('image is stale. Refreshing...')
                image = wd.find_elements(By.CLASS_NAME, 'athenaProductImageCarousel_image')[i]
            except Exception:
                print('Unexpected exception while getting image link.')
                image = wd.find_elements(By.CLASS_NAME, 'athenaProductImageCarousel_image')[i]
        variation_details[f'product_image_{i+1}'] = image_src
    return variation_details


def get_variation_misc_details(wd: webdriver.WebDriver, variation_details:dict[str, object], product_id: str):
    variation_details['variant_SKU'] = product_id
    variation_details['product_name'] = wd.find_element(By.CLASS_NAME, 'productName_title').get_attribute('textContent')
    try:
        variation_details['product_rating'] = float(wd.find_element(By.CLASS_NAME, 'productReviewStarsPresentational').get_attribute('aria-label').split(' ')[0])
    except NoSuchElementException:
        variation_details['product_rating'] = None
    try:
        variation_details['number_of_reviews'] = int(wd.find_element(By.CLASS_NAME, 'productReviewStars_numberOfReviews').text.split(' ')[0])
    except NoSuchElementException:
        variation_details['number_of_reviews'] = None
    variation_details['price'] = wd.find_element(By.CLASS_NAME, 'productPrice_price').text.removeprefix('£')
    try:
        wd.find_element(By.CLASS_NAME, 'productAddToBasket-soldOut')
        variation_details['in_stock'] = 'no'
    except NoSuchElementException:
        variation_details['in_stock'] = 'yes'
    return variation_details

def get_multi_size_details(wd: webdriver.WebDriver, product_details: dict[str, object]) -> list[dict[str, object]]:
    variations = []
    ids = [button.get_attribute("data-linked-product-id") for button in wd.find_elements(By.CLASS_NAME, 'athenaProductVariations_box')]
    for product_id in ids:
        button = wd.find_element(By.CSS_SELECTOR, f"button[data-linked-product-id='{product_id}']")
        variation_details = product_details.copy()
        is_selected = safe_get_element(button, By.CLASS_NAME, 'srf-hide')
        if is_selected is None:
            old_price = get_old_price(wd)
            wd.execute_script('arguments[0].click();', button)
            try:
                WebDriverWait(wd, 10).until(EC.staleness_of(old_price))
            except Exception:
                print(f'Could not find old price for url: "{product_details["product_url"]}"')
            button = wd.find_element(By.CSS_SELECTOR, f"button[data-linked-product-id='{product_id}']")
        variation_details = get_variation_misc_details(wd, variation_details, product_id)
        variation_details['size'] = button.text
        variation_details = get_variation_images(wd, variation_details)
        variations.append(variation_details)
    return variations

def get_id_from_url(url:str):
    base_name = os.path.basename(urlsplit(url).path)
    return base_name.split('.')[0].split('-')[0].strip()

def get_old_price(wd: webdriver.WebDriver):
    try:
        return wd.find_element(By.CLASS_NAME, 'productPrice_price')
    except NoSuchElementException:
        return wd.find_element(By.CLASS_NAME, 'productPrice_fromPrice')
    
def rgb_to_hex(rgb: list):
    return '#%02x%02x%02x' % (int(rgb[0]), int(rgb[1]), int(rgb[2]))

def get_multi_color_details(wd: webdriver.WebDriver, product_details: dict[str, object], product_type: str) -> list[dict[str, object]]:
    variations = []
    drop_down_list = wd.find_element(By.CLASS_NAME, 'athenaProductVariations_dropdown')
    select = Select(drop_down_list)
    for option, id in [(x.text, x.get_attribute('value')) for x in select.options if x.text.casefold() != 'Please choose...'.casefold()]:
        variation_details = product_details.copy()
        old_price = get_old_price(wd)
        select = Select(wd.find_element(By.CLASS_NAME, 'athenaProductVariations_dropdown'))
        select.select_by_visible_text(option)
        try:
            WebDriverWait(wd, 10).until(EC.staleness_of(old_price))
        except Exception:
            print(f'Could not find old price for url: "{product_details["product_url"]}"')
        variation_details = get_variation_images(wd, variation_details)
        product_id = get_id_from_url(variation_details['product_image_1'])
        variation_details = get_variation_misc_details(wd, variation_details, product_id)
        if product_type == ProductType.MULTI_COLOR:
            variation_type = 'color'
        elif product_type == ProductType.MULTI_SHADE:
            variation_type = 'shade'
        else:
            raise ValueError(f'Invalid product type: {product_type}')
        variation_details[variation_type] = option
        color = wd.find_element(By.CSS_SELECTOR, f"span[data-value-id='{id}']").value_of_css_property('background-color')
        color = Color.from_string(color).hex
        variation_details[f'{variation_type}_hex'] = color
        variations.append(variation_details)
    return variations


def get_product_details(wd:webdriver.WebDriver, urls: list[str]):
    df = pd.DataFrame()
    for url in urls:
        try:
            wd.get(url)
            product_details = {}
            product_variations = []
            brand_element = wd.find_element(By.CLASS_NAME, 'productBrandLogo_image')
            product_details['product_url'] = url
            product_details['brand_name'] = brand_element.get_attribute('title')
            product_details['brand_logo'] = brand_element.get_attribute('src')
            product_details['primary_SKU'] = get_id_from_url(url)
            for button in wd.find_elements(By.CLASS_NAME, 'productDescription_accordionControl'):
                try:
                    if not button.text:
                        continue
                    button_id = button.get_attribute("id")
                    is_expanded = button.get_attribute('aria-expanded')
                    if is_expanded == 'false':
                        wd.execute_script("arguments[0].click();", button)
                    description_content = wd.find_element(By.ID, button_id.replace('heading', 'content')).text
                    product_details[button.text] = description_content

                except ElementNotInteractableException:
                    print(f'cannot click element with id: {button_id}')
                except Exception  as e:
                    print(f'Unexpected error occurred: {traceback.format_exc()}')
            variation_label = safe_get_element(wd, By.CLASS_NAME, 'athenaProductVariations_dropdownLabel')
            if variation_label is not None:
                variation = variation_label.text.strip()
                if variation.casefold() in color_variation_tags:
                    product_details['product_type'] = ProductType.MULTI_COLOR
                    product_variations = get_multi_color_details(wd, product_details, ProductType.MULTI_COLOR)
                elif variation.casefold() in shade_variation_tags:
                    product_details['product_type'] = ProductType.MULTI_SHADE
                    product_variations = get_multi_color_details(wd, product_details, ProductType.MULTI_SHADE)
                elif variation.casefold() in size_variation_tags:
                    product_details['product_type'] = ProductType.MULTI_SIZE
                    product_variations = get_multi_size_details(wd, product_details)
                else:
                    print(f'Unknown variant type: {variation}')
            else:
                product_details['product_type'] = ProductType.SINGLE
                product_details = get_variation_images(wd, product_details)
                product_id = get_id_from_url(product_details['product_image_1'])
                product_details = get_variation_misc_details(wd, product_details, product_id)
                product_variations = [product_details]
            df = pd.concat([df, pd.DataFrame(product_variations)], ignore_index=True)
        except Exception as e:
            print(f'Unexpected error with trying to fetch data in url "{url}". \n{e}')
    return df

browser_options = options.Options()
browser_options.add_argument('-disable-notifications')
browser_options.add_experimental_option("prefs", {"profile.default_content_setting_values.cookies": 2})
# browser_options.add_argument('-headless')

color_variation_tags = [x.casefold() for x in ['colour:', 'color:']]
shade_variation_tags = [x.casefold() for x in ['shade:']]
size_variation_tags = [x.casefold() for x in ['size:']]

PRODUCT_LINKS = ["https://www.cultbeauty.co.uk/westman-atelier-eye-pods/13324061.html?affil=thgppc&countrySelected=Y",
                 "https://www.cultbeauty.co.uk/huda-beauty-lovefest-obsessions-eyeshadow-palette/13899183.html?affil=thgppc&countrySelected=Y",
                 'https://www.cultbeauty.co.uk/hindash-manifesto-lipstick-3.5g-various-shades/13798789.html?affil=thgppc&settingsSaved=Y&shippingcountry=GB&switchcurrency=GBP&countrySelected=Y',
                 'https://www.cultbeauty.co.uk/welleco-nourishing-protein-chocolate-refill/13314044.html?affil=thgppc&countrySelected=Y',
                 'https://www.cultbeauty.co.uk/thank-you-farmer-sun-project-water-sun-cream-spf50/13313863.html?affil=thgppc&countrySelected=Y',
                 'https://www.cultbeauty.co.uk/color-wow-travel-dream-coat-supernatural-spray-50ml/11870457.html',
                 'https://www.cultbeauty.com/huda-beauty-kayali-yum-pistachio-gelato-33-eau-de-parfum-intense-10ml/14272370.html']

# df = get_product_details(PRODUCT_LINKS, browser_options)
# df.head()

In [ ]:
with pd.option_context('display.max_columns', None, 'display.max_rows', None):
    display(df.head(100))

In [ ]:
from concurrent.futures import ProcessPoolExecutor
from concurrent.futures import as_completed

CATEGORY_LINKS = ['https://www.cultbeauty.com/body-wellbeing/tanning-suncare/shop-all.list',
                  'https://www.cultbeauty.com/skin-care.list',
                   'https://www.cultbeauty.com/make-up.list']
# CATEGORY_LINKS = ['https://www.cultbeauty.com/body-wellbeing/tanning-suncare/shop-all.list',
#                   'https://www.cultbeauty.com/skin-care.list',
#                   'https://www.cultbeauty.com/make-up.list',
#                   'https://www.cultbeauty.com/hair-care.list',
#                   'https://www.cultbeauty.com/body-wellbeing.list',
#                   'https://www.cultbeauty.com/fragrance.list',
#                   'https://www.cultbeauty.com/gifts.list',
#                   'https://www.cultbeauty.com/minis.list',
#                   'https://www.cultbeauty.com/sale.list',
#                   'https://www.cultbeauty.com/men.list']

def get_category_links(browser_options: options.Options, url):
    with webdriver.WebDriver(browser_options) as wd:
        page = 1
        wd.get(f'{url}?pageNumber={page}')
        product_details = pd.DataFrame()
        while True:
            product_links = list(set([x.find_element(By.CLASS_NAME, 'productBlock_link').get_attribute('href') for x in wd.find_elements(By.CLASS_NAME, 'productBlock_itemDetails_wrapper')]))
            product_details = pd.concat([product_details, get_product_details(wd, product_links)])
            try:
                next_page_button = wd.find_element(By.CSS_SELECTOR, 'button.responsivePaginationNavigationButton.paginationNavigationButtonNext')
            except NoSuchElementException:
                print(f'Could not find next button in: "{url}. Page: {page}"')
                return product_details
            if next_page_button.get_attribute('disabled') == 'true':
                break
            page += 1
        return product_details

def main():
    all_links = pd.DataFrame()
    with ProcessPoolExecutor(max_workers = 3) as executor:
        results = executor.map(get_category_links, [browser_options for link in CATEGORY_LINKS],CATEGORY_LINKS)
    for result in results:
        all_links = pd.concat([all_links, result])
    print(all_links.shape)
    all_links.head()

if __name__ == '__main__':
    main()


In [ ]:
import pandas as pd

df = pd.read_excel('./test_cult_beauty.xlsx')

In [ ]:
df.shape

In [ ]:
with pd.option_context("display.max_columns", None, 'max_colwidth', None):
    display(df.loc[df['variant_SKU'] == 'default'])

In [ ]:
with pd.option_context("display.max_rows", None):
    display(df['variant_SKU'].value_counts(sort=True, ascending=False))

In [116]:
import pandas as pd

df = pd.read_excel('./test_cult_beauty_without_duplicates.xlsx')

In [38]:
df.loc[(~pd.isna(df['option'])) & df['option'].str.contains('refill', case=False)]
df.dr

,product_url,brand_name,brand_logo,primary_SKU,Why It's Cult,Description,How to Use,Full Ingredients List,Product Details,product_type,...,in_stock,Cult Conscious,size,shade,shade_hex,color,color_hex,About the Range,serialized_primary_SKU,is_variant_of
470,https://www.cultbeauty.com/skingredients-skin-...,NaN,NaN,13322150,Created by dermal facialist and skin nerd Jenn...,Skin Shield SPF 50 PA+++ is your moisturising ...,Use every single day in the AM. Apply ½ a teas...,"Aqua (Water), Zinc Oxide (12%), C12-15 Alkyl B...",Volume:\n73ml\nBrand:\nSkingredients,multi-option,...,no,Transparency lies at the heart of our philosop...,NaN,NaN,NaN,NaN,NaN,NaN,13322150-2,13322150
542,https://www.cultbeauty.com/shiseido-exclusive-...,Shiseido,https://static.thcdn.com/design-assets/images/...,13497033,"Employing their years of scientific know-how, ...",This lightly floral scented moisturiser covers...,"Use every morning, after cleansing and serum. ...",Water(Aqua/Eau)･Alcohol Denat.･Dimethicone･Gly...,Volume:\n50ml\nBrand:\nShiseido,multi-option,...,yes,NaN,NaN,NaN,NaN,NaN,NaN,NaN,13497033-3,13497033
803,https://www.cultbeauty.com/byoma-balancing-fac...,BYOMA,https://static.thcdn.com/design-assets/images/...,13521365,"The next-gen, dermatologically-loved and appro...","Spritz and spray your way to a balanced, healt...",Shake well before each use and mist onto clean...,"Aqua, Butylene Glycol, Glycerin, Propanediol, ...",Range:\nMain\nBrand:\nBYOMA\nVolume:\n100ml,multi-option,...,yes,Transparency lies at the heart of our philosop...,NaN,NaN,NaN,NaN,NaN,NaN,13521365-3,13521365
817,https://www.cultbeauty.com/byoma-moisturising-...,BYOMA,https://static.thcdn.com/design-assets/images/...,13521362,A moisturising cocktail of barrier boosting no...,This ultra-lightweight and deliciously creamy ...,Apply 1-2 pumps on cleansed skin following ser...,"Aqua, Glycerin, Caprylic/Capric Triglyceride, ...",Range:\nMain\nBrand:\nBYOMA\nVolume:\n50ml,multi-option,...,yes,Transparency lies at the heart of our philosop...,NaN,NaN,NaN,NaN,NaN,NaN,13521362-3,13521362
822,https://www.cultbeauty.com/byoma-moisturising-...,BYOMA,https://static.thcdn.com/design-assets/images/...,13521363,Environmentally-conscious and designed with ev...,Rich in skin-loving moisture magnets that melt...,Apply 1-2 pumps on cleansed skin following ser...,"Aqua, Glycerin, Caprylic/Capric Triglyceride, ...",Range:\nMain\nBrand:\nBYOMA\nVolume:\n50ml,multi-option,...,yes,Transparency lies at the heart of our philosop...,NaN,NaN,NaN,NaN,NaN,NaN,13521363-3,13521363
853,https://www.cultbeauty.com/byoma-brightening-s...,BYOMA,https://static.thcdn.com/design-assets/images/...,13521360,"Knowledge is power and BYOMA, being the scient...",This super-charged and multitasking BYOMA Brig...,"Apply 4-6 drops to cleansed skin, gently press...","Aqua, Methylpropanediol, Niacinamide, 1,2-Hexa...",Range:\nMain\nBrand:\nBYOMA\nVolume:\n30ml,multi-option,...,yes,Transparency lies at the heart of our philosop...,NaN,NaN,NaN,NaN,NaN,NaN,13521360-3,13521360
878,https://www.cultbeauty.com/byoma-creamy-jelly-...,BYOMA,https://static.thcdn.com/design-assets/images/...,13521364,"Taking a ‘less is more’ approach, cleansing yo...",Are you ready for this jelly? The ultra hydrat...,Massage 1-2 pumps onto damp skin in slow circu...,"Aqua, Cocamidopropyl Betaine, Disodium Laureth...",Range:\nMain\nBrand:\nBYOMA\nVolume:\n175ml,multi-option,...,yes,Transparency lies at the heart of our philosop...,NaN,NaN,NaN,NaN,NaN,NaN,13521364-3,13521364
880,https://www.cultbeauty.com/byoma-hydrating-ser...,BYOMA,https://static.thcdn.com/design-assets/images/...,13521359,Ever felt like you’ve overexfoliated or overtr...,"This ultra-lightweight, moisture boosting seru...","Apply 4-6 drops to cleansed skin, gently press...","Aqua, Glycerin, Butylene Glycol, Squalane, Sod...",Range:\nMain\nBrand:\nBYOMA\nVolume:\n30ml,multi-option,...,yes,Transparency lies at the heart of our philosop...,NaN,NaN,NaN,NaN,NaN,NaN,13521359-3,13521359
887,ht

In [39]:
def first_not_null(names: pd.Series):
    return next((x for x in names if not pd.isna(x)), pd.NA)
combined_variants = df[['option', 'color', 'size', 'shade']].apply(first_not_null, axis= 1)

In [47]:
df.loc[(~pd.isna(combined_variants)) & combined_variants.str.contains('€', case=False)]

,product_url,brand_name,brand_logo,primary_SKU,Why It's Cult,Description,How to Use,Full Ingredients List,Product Details,product_type,...,in_stock,Cult Conscious,size,shade,shade_hex,color,color_hex,About the Range,serialized_primary_SKU,is_variant_of
13496,https://www.cultbeauty.com/customer-gift-vouch...,Cult Beauty,https://static.thcdn.com/design-assets/images/...,13466897,NaN,NaN,NaN,NaN,Brand:\nCult Beauty,multi-option,...,yes,NaN,NaN,NaN,NaN,NaN,NaN,NaN,13466897-2,13466897
13497,https://www.cultbeauty.com/customer-gift-vouch...,Cult Beauty,https://static.thcdn.com/design-assets/images/...,13466897,NaN,NaN,NaN,NaN,Brand:\nCult Beauty,multi-option,...,yes,NaN,NaN,NaN,NaN,NaN,NaN,NaN,13466897-3,13466897
13498,https://www.cultbeauty.com/customer-gift-vouch...,Cult Beauty,https://static.thcdn.com/design-assets/images/...,13466897,NaN,NaN,NaN,NaN,Brand:\nCult Beauty,multi-option,...,yes,NaN,NaN,NaN,NaN,NaN,NaN,NaN,13466897-4,13466897
13499,https://www.cultbeauty.com/customer-gift-vouch...,Cult Beauty,https://static.thcdn.com/design-assets/images/...,13466897,NaN,NaN,NaN,NaN,Brand:\nCult Beauty,multi-option,...,yes,NaN,NaN,NaN,NaN,NaN,NaN,NaN,13466897-5,13466897
13500,https://www.cultbeauty.com/customer-gift-vouch...,Cult Beauty,https://static.thcdn.com/design-assets/images/...,13466897,NaN,NaN,NaN,NaN,Brand:\nCult Beauty,multi-option,...,yes,NaN,NaN,NaN,NaN,NaN,NaN,NaN,13466897-6,13466897
13501,https://www.cultbeauty.com/customer-gift-vouch...,Cult Beauty,https://static.thcdn.com/design-assets/images/...,13466897,NaN,NaN,NaN,NaN,Brand:\nCult Beauty,multi-option,...,yes,NaN,NaN,NaN,NaN,NaN,NaN,NaN,13466897-7,13466897


In [53]:
why_its_cult = "Why It's Cult"
df[[why_its_cult, 'Description']].apply(lambda x: f"Description:\n{x['Description']}\nWhy It's On SIIN:\n{x[why_its_cult]}", axis=1)
df.drop(why_its_cult, axis=1)

,product_url,brand_name,brand_logo,primary_SKU,Description,How to Use,Full Ingredients List,Product Details,product_type,option,...,in_stock,Cult Conscious,size,shade,shade_hex,color,color_hex,About the Range,serialized_primary_SKU,is_variant_of
0,https://www.cultbeauty.com/coola-spray-spf50-1...,COOLA,https://static.thcdn.com/design-assets/images/...,12872390,"Great for long days in the sun, COOLA’s quick ...",Apply liberally 15 minutes before sun exposure...,Fragrance free: Alcohol Denat. (Sd Alcohol 40-...,Range:\nSPF\nVolume:\n177ml\nBrand:\nCOOLA,multi-option,Fragrance Free,...,yes,NaN,NaN,NaN,NaN,NaN,NaN,NaN,12872390-2,12872390
1,https://www.cultbeauty.com/coola-spray-spf50-1...,COOLA,https://static.thcdn.com/design-assets/images/...,12872390,"Great for long days in the sun, COOLA’s quick ...",Apply liberally 15 minutes before sun exposure...,Fragrance free: Alcohol Denat. (Sd Alcohol 40-...,Range:\nSPF\nVolume:\n177ml\nBrand:\nCOOLA,multi-option,Guava Mango,...,yes,NaN,NaN,NaN,NaN,NaN,NaN,NaN,12872390-3,12872390
2,https://www.cultbeauty.com/by-terry-baume-de-r...,By Terry,https://static.thcdn.com/design-assets/images/...,11307246,"Skin care with colour, By Terry is a pioneerin...",Apply Baume de Rose directly to lips with your...,"Active Ingredients: Octinoxate (7.5%), Avobenz...",Brand:\nBy Terry\nVolume:\n10g,single,NaN,...,yes,NaN,NaN,NaN,NaN,NaN,NaN,NaN,11307246-2,11307246
3,https://www.cultbeauty.com/megababe-space-bar-...,Megababe,https://static.thcdn.com/design-assets/images/...,13798836,"Don’t sweat the small stuff — or if you do, le...",Use every time you shower for deeply cleansed ...,"Sodium Cocoate, Water, Glycerin, Coconut Acid,...",Brand:\nMegababe,single,NaN,...,yes,NaN,NaN,NaN,NaN,NaN,NaN,NaN,13798836-2,13798836
4,https://www.cultbeauty.com/st.-tropez-self-tan...,St. Tropez,https://static.thcdn.com/design-assets/images/...,12013394,"Deliciously tropical scented, this new Self Ta...",Step 1: Exfoliate 24 hours prior to use. Moist...,"Aqua (Water/ Eau), Dihydroxyacetone, PEG-40 Hy...",Range:\nSelf Tan\nBrand:\nSt. Tropez\nVolume:\...,single,NaN,...,yes,Transparency lies at the heart of our philosop...,NaN,NaN,NaN,NaN,NaN,NaN,12013394-2,12013394
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
14039,https://www.cultbeauty.com/allies-of-skin-mult...,Allies of Skin,https://static.thcdn.com/design-assets/images/...,14510973,Think of the Multi Peptides & GF Advanced Lift...,Shake well before every use. After cleansing a...,"Aqua (Water), Glycerin, Heptyl Glucoside, Isod...",Brand:\nAllies of Skin\nVolume:\n30ml,single,NaN,...,no,NaN,NaN,NaN,NaN,NaN,NaN,NaN,14510973-2,14510973
14040,https://www.cultbeauty.com/baxter-of-californi...,Baxter of California,https://static.thcdn.com/design-assets/images/...,11477433,"Established in 1965, Baxter of California is a...",Apply deodorant to clean underarms.,"Propylene Glycol, Aqua / Water, Sodium Stearat...",Brand:\nBaxter of California,multi-size,NaN,...,no,NaN,Full Size,NaN,NaN,NaN,NaN,NaN,11477433-2,11477433
14041,https://www.cultbeauty.com/baxter-of-californi...,Baxter of California,https://static.thcdn.com/design-assets/images/...,10363811,"Established in 1965, Baxter of California is a...","Massage onto damp face and neck in a light, ci...","Aqua / Water, Zea Mays Kernel Meal / Corn Kern...",Brand:\nBaxter of California\nVolume:\n120ML,single,NaN,...,no,NaN,NaN,NaN,NaN,NaN,NaN,NaN,10363811-2,10363811
14042,https://www.cultbeauty.com/baxter-of-californi...,Baxter of California,https://static.thcdn.com/design-assets/images/...,11474278,"Established in 1965, Baxter of California is a...","After shaving, apply a small amount to the fac...","Aqua / Water, Glycerin, Butylene Glycol, Methy...",Brand:\nBaxter of California\nVolume:\n120ml,single,NaN,...,no,NaN,NaN,NaN,NaN,NaN,NaN,NaN,11474278-2,11474278


In [100]:
import re
test = """

Charlotte Tilbury's award-winning Magic Cream was created backstage to transform models' skin in an instant. It soon became so cult that Charlotte was compelled to 'bottle' her secret formula; a moisturising cocktail of plumping, firming and line-lessening complexes that rapidly bestows a gorgeous, runway-ready glow. Pick from a sumptuous glass full size (50ml), or sized-to-try, travel-friendly plastic sizes (30ml) and (15ml), so you're never without this go-to cream again. Magic by name and by nature, this fan-favourite formula is now housed in a sustainably minded, refillable jar... and available in limited edition packaging to bring you luck this Lunar New Year. Simply replenish your stores with Charlotte's Magic Cream Refill.

WE REGRET WE CAN'T SHIP CHARLOTTE TILBURY TO THE MIDDLE EAST
"""

In [126]:
re.sub(r'we regret.+(?:middle east|bahrain)', '', None, flags=re.IGNORECASE)

TypeError: expected string or bytes-like object, got 'NoneType'

In [ ]:
def ship_to_bahrain(text: str, pattern = r'we regret.+(?:middle east|bahrain)'):
    if pd.isna(text):
        return 'no'
    found = re.search(pattern, text, flags=re.IGNORECASE)
    if found is None:
        return 'yes'

    return 'no'

In [106]:
mask = df.loc[~pd.isna(df[why_its_cult]) & df[why_its_cult].str.contains(r'we regret.+(?:middle east|bahrain)', case=False, regex=True)].index
df[why_its_cult].transform(ship_to_bahrain).value_counts()

Index([  736,   782,   783,   797,   798,   856,   927,   951,   952,   953,
       ...
        7718,  8226,  9137,  9741,  9879, 10143, 10144, 10145, 10200, 12556],
      dtype='int64', length=187)

In [112]:
def remove_pattern(text: str, pattern = r'we regret.+(?:middle east|bahrain)'):
    if pd.isna(text):
        return text
    found = re.search(pattern, text, flags=re.IGNORECASE)
    if found is None:
        return text

    return re.sub(pattern, '', text, flags=re.IGNORECASE)

In [123]:
df = df.applymap(lambda x: x.strip() if isinstance(x, str) else x)

In [124]:
df['Description']

0        Great for long days in the sun, COOLA’s quick ...
1        Great for long days in the sun, COOLA’s quick ...
2        Skin care with colour, By Terry is a pioneerin...
3        Don’t sweat the small stuff — or if you do, le...
4        Deliciously tropical scented, this new Self Ta...
                               ...                        
14039    Think of the Multi Peptides & GF Advanced Lift...
14040    Established in 1965, Baxter of California is a...
14041    Established in 1965, Baxter of California is a...
14042    Established in 1965, Baxter of California is a...
14043    Established in 1965, Baxter of California is a...
Name: Description, Length: 14044, dtype: object

In [122]:
df[why_its_cult].transform(remove_pattern).loc[~pd.isna(df[why_its_cult]) & df[why_its_cult].str.endswith('\n')]

Series([], Name: Why It's Cult, dtype: object)

In [45]:
mask = df.loc[(~pd.isna(combined_variants)) & combined_variants.str.contains('€', case=False)].index
df.drop(mask).loc[(~pd.isna(combined_variants)) & combined_variants.str.contains('€', case=False)]

,product_url,brand_name,brand_logo,primary_SKU,Why It's Cult,Description,How to Use,Full Ingredients List,Product Details,product_type,...,in_stock,Cult Conscious,size,shade,shade_hex,color,color_hex,About the Range,serialized_primary_SKU,is_variant_of


In [144]:
def remove_brand_name(row: pd.Series):
    if (any((pd.isna(x) for x in row.values))):
        return row['product_name']
    if not (row['product_name'].casefold().startswith(row['brand_name'].casefold())):
        return row['product_name']
    new_product_name = row['product_name'].removeprefix(row['brand_name']).strip()
    return new_product_name

In [145]:
df[['brand_name', 'product_name']].apply(remove_brand_name, axis=1)

0                                        Spray SPF50 177ml
1                           Guava Mango Spray SPF 50 177ml
2                        Baume De Rose Lip Balm SPF 15 10g
3          Space Bar Detoxifying Charcoal Underarm Bar 99g
4                                Self Tan Purity Gel 200ml
                               ...                        
14039    Multi Peptides and GF Advanced Lifting Serum 30ml
14040             Aluminium and Alcohol Free Deodorant 75g
14041                                   Facial Scrub 120ml
14042                               After Shave Balm 120ml
14043                      Super Close Shave Formula 240ml
Length: 14044, dtype: object

In [ ]:

df[['option', 'color', 'size', 'shade', 'in_stock']].loc[(df['in_stock'] == 'yes') & (~pd.isna(combined_variants.str)) & (combined_variants.str.endswith('- Out of stock'))].transform

In [ ]:
df.iloc[10170][['product_url', 'price']]

In [ ]:
df.drop_duplicates('variant_SKU', inplace=True, ignore_index=True)

In [ ]:
import os
from urllib.parse import urlsplit
def get_id_from_url(url:str):
    base_name = os.path.basename(urlsplit(url).path)
    return base_name.split('.')[0].split('-')[0].strip()
df['primary_SKU'] = df['primary_SKU'].transform(get_id_from_url)

In [ ]:
def create_serialized_sku(group:pd.Series, mask):
    count = 2
    serialized_skus = []
    for idx, row in group.items():
        if mask[idx]:
            serialized_skus.append((f"{row}-1", pd.NA))
        else:
            serialized_skus.append((f"{row}-{count}", row))
            count += 1
    return pd.Series(serialized_skus, index=group.index)

mask = df['primary_SKU'] == df['variant_SKU']
transform = df.groupby('primary_SKU')['primary_SKU'].transform(create_serialized_sku, mask)
df[['serialized_primary_SKU', 'is_variant_of']] = pd.DataFrame(transform.to_list(), columns=['serialized_primary_SKU', 'is_variant_of']
                                                                , index=transform.index)

In [ ]:
import re
df['price'] = df['price'].transform(lambda x: re.sub(r'[^\d.]', '', x))

In [ ]:
df.shape

In [ ]:
def order_serialized_columns(columns: list[str], regex = r'_(\d+)'):
    ordered_columns = []
    groups = {}
    for i, column in enumerate(columns):
        index = re.search(regex, column)
        if index is None or index.group(1) is None:
            ordered_columns.append(column)
            continue
        index = int(index.group(1))
        group_name = re.sub(regex, '', column)
        if group_name not in groups:
            groups[group_name] = {'starting_index': i, 'names': [{'index':index, 'name':column}]}
        else:
            groups[group_name]['names'].append({'index':index, 'name':column})
            if i < groups[group_name]['starting_index']:
                groups[group_name]['starting_index'] = i
    for group in groups.values():
        group['names'] = sorted(group['names'], key=lambda d: d['index'], reverse=True) 

        for name in group['names']:
            ordered_columns.insert(group['starting_index'], name['name'])

    return ordered_columns
df = df.reindex(order_serialized_columns(df.columns), axis=1)

In [ ]:
df.dropna(axis=1, how='all', inplace=True)

In [ ]:
df.count()

In [ ]:

df.to_excel('./test_cult_beauty_without_duplicates.xlsx', index=False)

In [ ]:
from selenium.webdriver.chrome import webdriver, options
from selenium.webdriver.support.select import Select, By, WebElement
import time
from selenium.common.exceptions import NoSuchElementException, ElementNotInteractableException, StaleElementReferenceException, TimeoutException
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.support import expected_conditions as EC

browser_options = options.Options()
browser_options.add_argument('-disable-notifications')
# browser_options.add_experimental_option("prefs", {"profile.default_content_setting_values.cookies": 2})

def wait_for_presence_get(wd: webdriver.WebDriver, by: By, value: str, wait_for: int = 2):
    try:
        WebDriverWait(wd, wait_for).until(EC.visibility_of_element_located((by, value)))
    except TimeoutException:
        return None
    return wd.find_element(by, value)

def click_element_refresh_stale(wd: webdriver.WebDriver, element: WebElement, by: By, locator: str, index = None):
    while True:
        try:
            wd.execute_script("arguments[0].click();", element)
            return element
        except Exception:
            print('Could not click element. Refreshing...')
            if index is None:
                element = wd.find_element(by, locator)
            else:
                element = wd.find_elements(by, locator)[index]

with webdriver.WebDriver(browser_options) as wd:
    wd.maximize_window()
    wd.get('https://www.cultbeauty.com/body-wellbeing/tanning-suncare/shop-all.list')
    time.sleep(5)
    settings_button = wait_for_presence_get(wd ,By.CLASS_NAME, 'responsiveSubMenu_sessionSettings')
    wd.execute_script("arguments[0].click();", settings_button)
    currency_select_button = wait_for_presence_get(wd ,By.CLASS_NAME, 'sessionSettings_currencySelect')
    Select(currency_select_button).select_by_visible_text('£ (GBP)')
    save_button = wait_for_presence_get(wd ,By.CLASS_NAME, 'sessionSettings_saveButton')
    wd.execute_script("arguments[0].click();", save_button)
    time.sleep(5)
